## MyID3 and MyBaggingID3 implementation

In [20]:
# import libraries
import numpy as np


def compute_entropy(y):
    """
    Computes the entropy for
    Args:
       y (ndarray): Numpy array indicating whether each example at a node is
           positive (`1`) or negative (`0`)
    Returns:
        entropy (float): Entropy at that node
    """
    if len(y) == 0:
        return 0
    p1 = np.sum(y) / len(y)
    if p1 == 0 or p1 == 1:
        return 0
    h = -p1 * np.log2(p1) - (1 - p1) * np.log2(1 - p1)
    return h


def split_dataset(X, node_indices, feature):
    """
    Splits the data at the given node into
    left and right branches

    Args:
        X (ndarray):             Data matrix of shape(n_samples, n_features)
        node_indices (list):  List containing the active indices. I.e, the samples being considered at this step.
        feature (int):           Index of feature to split on

    Returns:
        left_indices (list): Indices with feature value == 1
        right_indices (list): Indices with feature value == 0
    """
    left_indices = []
    right_indices = []
    ## loop over the list node_indices (the train set)
    for index in node_indices:
        if X[index][feature] == 0:
            right_indices.append(index)
        else:
            left_indices.append(index)
    return left_indices, right_indices


def compute_information_gain(X, y, node_indices, feature):
    """
    Compute the information of splitting the node on a given feature

    Args:
        X (ndarray):            Data matrix of shape(n_samples, n_features)
        y (array like):         list or ndarray with n_samples containing the target variable
        node_indices (ndarray): List containing the active indices. I.e, the samples being considered in this step.
        feature (int):           Index of feature to split on
    Returns:
        cost (float):        Cost computed"""
    # create left and right branches
    left_indices, right_indices = split_dataset(X, node_indices, feature)
    w_l = len(left_indices) / len(node_indices)
    w_r = len(right_indices) / len(node_indices)
    # calculate the y in the training data
    X_train = []
    y_train = []
    for index in node_indices:
        X_train.append(X[index])
        y_train.append(y[index])
    # calculate the y of the left and right branches
    y_left = []
    y_right = []
    for index in left_indices:
        y_left.append(y[index])
    for index in right_indices:
        y_right.append(y[index])
    ig = compute_entropy(y_train) - (w_l * compute_entropy(y_left) + w_r * compute_entropy(y_right))
    return ig


def get_best_split(X, y, node_indices):
    """
    Returns the optimal feature and threshold value
    to split the node data

    Args:
        X (ndarray):            Data matrix of shape(n_samples, n_features)
        y (array like):         list or ndarray with n_samples containing the target variable
        node_indices (ndarray): List containing the active indices. I.e, the samples being considered in this step.
    Returns:
        best_feature (int):     The index of the best feature to split
    """
    feature_list = np.arange(len(X[0])).tolist()
    best_result = 0
    best_feature = "nan"
    # loop ove feature list in order to see whch feature have the best split
    for feature in feature_list:
        ig = compute_information_gain(X, y, node_indices, feature)
        if ig > best_result:
            best_result = ig
            best_feature = feature
    return best_feature


def predict(X, y, node_indices):
    """

    :param X: matrix
    :param y: output of matrix
    :param node_indices: the indices which exsits in the node
    :return:
     :param prediciton: what is th output of the node
    """
    # define count varaible which count the number of the 1/0
    zero = 0
    one = 0
    # loop over the indices in the node
    for i in node_indices:
        if y[i] == 0:
            zero += 1
        else:
            one += 1
    # decide who is biggger (one or zero)
    result_pred = 0
    if one > zero:
        result_pred = 1, one / (zero + one)
    prob_0 = zero / (zero + one)
    prob_1 = one / (zero + one)
    return result_pred, prob_0, prob_1


def max_sample_n_features(X, y, max_samples, max_features):
    # define the percentage of rows and columns to keep
    row_percent_to_keep = max_samples
    col_percent_to_keep = max_features

    # compute the number of rows and columns to keep
    n_rows_to_keep = int(X.shape[0] * row_percent_to_keep)
    n_cols_to_keep = int(X.shape[1] * col_percent_to_keep)
    # generate random row and column indices for the rows and columns to keep
    row_indices = np.random.choice(X.shape[0], size=n_rows_to_keep, replace=True)
    col_indices = np.random.choice(X.shape[1], size=n_cols_to_keep, replace=False)
    # take the selected rows and columns from X_train and y_train
    selected_X_train = X[row_indices][:, col_indices]
    selected_y_train = y[row_indices]
    return selected_X_train, selected_y_train, col_indices


class Node:
    def __init__(self, X, y, node_indices, terminal=False, depth=0, right=None, left=None):
        """_summary_

        Args:
            X (array): matrix 2d array
            y (array): aray
            node_indices (array): array of the relevenat rows
            terminal (bool, optional): flag if it  final node. Defaults to False.
            depth (int, optional): the depth of the node. Defaults to 0.
            right (node, optional): root. Defaults to None.
            left (node, optional): root. Defaults to None.
        """
        self.left = left
        self.right = right
        self.terminal = terminal
        self.depth = depth
        self.X = X
        self.y = y
        self.node_indices = node_indices
        self.feature = get_best_split(X, y, node_indices)
        self.predict, self.predict_probab_0, self.predict_probab_1 = predict(X, y, node_indices)
        if self.feature != "nan":
            self.split()
        else:
            self.terminal = True

    def split(self):
        """
        split the trre into 2 new nodes (if there's option to split)
        """
        self.left, self.right = split_dataset(self.X, self.node_indices, self.feature)
        if len(self.left) == 0 or len(self.right) == 0:
            self.terminal = True
        self.left = Node(self.X, self.y, self.left, self.terminal, depth=(self.depth + 1))
        self.right = Node(self.X, self.y, self.right, self.terminal, depth=(self.depth + 1))


class MyID3:
    def __init__(self, max_depth=None):
        """
        Initial function of the class MYID3
        Args:
            max_depth (int, optional): the maximum depth of the required
                tree. Defaults to None.
        """
        self.max_depth = max_depth
        self.node_tree = None

    def fit(self, X, y):
        """
        Args:
        X (ndarray):            Data matrix of shape(n_samples, n_features)
        y (array like):         list or ndarray with n_samples containing the target variable
        """
        node_indices = np.arange(0, len(X), 1)
        self.node_tree = Node(X, y, node_indices)

    def predict(self, X):
        """
        Args:
            X (ndarray):            Data matrix of shape(n_samples, n_features)
        Returns:
            Prob (ndarray)                of shape (n_samples, n_classes)
        """
        y_prediced = []
        # loop over the X matrix, and check each sample
        for x in X:
            temp_node = self.node_tree
            while self.max_depth != temp_node.depth and temp_node.terminal == False:
                if x[temp_node.feature] == 1:
                    temp_node = temp_node.left
                else:
                    temp_node = temp_node.right
            if type(temp_node.predict) is not int:
                y_prediced.append(round(temp_node.predict[0]))
            else:
                y_prediced.append(round(temp_node.predict))
        return np.array(y_prediced)

    def predict_proba(self, X):
        """
        Args:
            X (ndarray):            Data matrix of shape(n_samples, n_features)
        Returns:
            Prob (ndarray)           of shape (n_samples, n_classes)
        """
        y_prediced_prob = []
        for x in X:
            predict_0_1 = []
            temp_node = self.node_tree
            while self.max_depth != temp_node.depth and temp_node.terminal == False:
                if x[temp_node.feature] == 1:
                    temp_node = temp_node.left
                else:
                    temp_node = temp_node.right
            predict_0_1.append(round(temp_node.predict_probab_0))
            predict_0_1.append(round(temp_node.predict_probab_1))
            y_prediced_prob.append(predict_0_1)
        return np.array(y_prediced_prob)


class MyBaggingID3:
    def __init__(self, max_depth=None, n_estimators=2, max_samples=1, max_features=1):
        """_summary_

        Args:
            max_depth (int, optional): _description_. Defaults to None.
            n_estimators (int, optional): The number of ID3 in the ensemble_. Defaults to 2.
            max_samples (int, optional):  A float num that represents the fraction of samples to draw from X  with replacement  to train each base estimator. Defaults to 1.
            max_features (int , optional): A float num (between 0 to 1) that represents the fraction of features to draw from X without replacement. Defaults to 1.
        """
        self.max_depth = max_depth
        self.n_estimators = n_estimators
        self.max_samples = max_samples
        self.max_features = max_features
        self.forest = []
        self.features = []

    def fit(self, X, y):
        """
        Args:
        X (ndarray):            Data matrix of shape(n_samples, n_features)
        y (array like):         list or ndarray with n_samples containing the target variable
        """
        node_indices = np.arange(0, len(X), 1)
        # create new x train any y train according to the hyperparameters
        # create forest ( list of trees)
        for tree in range(self.n_estimators):
            X_s, y_s, col_indices = max_sample_n_features(X, y, self.max_samples, self.max_features)
            tree = MyID3(max_depth=self.max_depth)
            tree.fit(X_s, y_s)
            self.forest.append(tree)
            self.features.append(col_indices)

    def predict(self, X):
        """
        Args:
            X (ndarray):            Data matrix of shape(n_samples, n_features)
        Returns:
            Prob (ndarray)                of shape (n_samples, n_classes)
        """
        # loop over each sample in X matrix
        y_prediced = []
        for x in X:
            tree_preidct = []
            i = 0
            # loop over each tree in the forest list
            for tree in self.forest:
                col = self.features[i]
                i += 1
                temp_node = tree.node_tree
                x_res = x[col]
                while self.max_depth != temp_node.depth and temp_node.terminal == False:
                    if x_res[temp_node.feature] == 1:
                        temp_node = temp_node.left
                    else:
                        temp_node = temp_node.right
                # insert the result of each tree to list
                tree_preidct.append(temp_node.predict)

            # each  the final result to the list, the final result by majority voting
            pred = max(set(tree_preidct), key=tree_preidct.count)
            if type(pred) is not int:
                y_prediced.append(pred[0])
            else:
                y_prediced.append(pred)
        return np.array(y_prediced)

    def predict_proba(self, X):
        """
        Args:
            X (ndarray):            Data matrix of shape(n_samples, n_features)
        Returns:
            Prob (ndarray)           of shape (n_samples, n_classes)
        """
        # loop over each sample in the matrix
        y_prediced_prob = []
        for x in X:
            # loop over each tree in the forest list
            tree_predict_prob = []
            i = 0
            for tree in self.forest:
                predict_0_1 = []
                col = self.features[i]
                i += 1
                temp_node = tree.node_tree
                x_res = x[col]
                while self.max_depth != temp_node.depth and temp_node.terminal == False:
                    if x_res[temp_node.feature] == 1:
                        temp_node = temp_node.left
                    else:
                        temp_node = temp_node.right
                # inset each result of the tree to the tree_predict_prob
                predict_0_1.append(temp_node.predict_probab_0)
                predict_0_1.append(temp_node.predict_probab_1)
                tree_predict_prob.append(predict_0_1)
            # calclualte the avearge of the forest over each sample
            y_prediced_prob.append(np.mean(np.array(tree_predict_prob), axis=0))
        return np.array(y_prediced_prob)

## Evaluation Report

In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import RepeatedKFold
import time
import matplotlib.pyplot as plt
import wandb

# ***Tic Tac Toe dataset***
"""
Number of Instances: 958
Number of Attributes: 10
Attribute values: x=player x has taken, o=player o has taken, b=blank
Class Distribution: About 65.3% are positive (i.e., wins for "x")
No missing values
"""
DF1 = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/tic-tac-toe/tic-tac-toe.data",
    header=None,
)
DF1.columns = [
    "top-left-square",
    "top-middle-square",
    "top-right-square",
    "middle-left-square",
    "middle-middle-square",
    "middle-right-square",
    "bottom-left-square",
    "bottom-middle-square",
    "bottom-right-square",
    "Class",
]


# ***Lymphography dataset***
"""
Number of Instances: 148
Number of Attributes: 19
Attribute values: 
    1. class: normal find, metastases, malign lymph, fibrosis
    2. lymphatics: normal, arched, deformed, displaced
    3. block of affere: no, yes
    4. bl. of lymph. c: no, yes
    5. bl. of lymph. s: no, yes
    6. by pass: no, yes
    7. extravasates: no, yes
    8. regeneration of: no, yes
    9. early uptake in: no, yes
   10. lym.nodes dimin: 0-3
   11. lym.nodes enlar: 1-4
   12. changes in lym.: bean, oval, round
   13. defect in node: no, lacunar, lac. marginal, lac. central
   14. changes in node: no, lacunar, lac. margin, lac. central
   15. changes in stru: no, grainy, drop-like, coarse, diluted, reticular, 
                        stripped, faint, 
   16. special forms: no, chalices, vesicles
   17. dislocation of: no, yes
   18. exclusion of no: no, yes
   19. no. of nodes in: 0-9, 10-19, 20-29, 30-39, 40-49, 50-59, 60-69, >=70
Class Distribution: 54.72% metastases, 41.21% malign lymph, 2.7% fibrosis, 1.3% normal find
No missing values
"""
DF2 = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/lymphography/lymphography.data",
    header=None,
)
DF2.columns = [
    "Class",
    "lymphatics",
    "block of affere",
    "bl. of lymph. c",
    "bl. of lymph. s",
    "by pass",
    "extravasates",
    "regeneration of",
    "early uptake in",
    "lym.nodes dimin",
    "lym.nodes enlar",
    "changes in lym.",
    "defect in node",
    "changes in node",
    "changes in stru",
    "special forms",
    "dislocation of",
    "exclusion of no",
    "no. of nodes in",
]

temp_ind = DF2[DF2["Class"].isin([1, 4])].index
DF2.drop(temp_ind, inplace=True)
cols = DF2.columns.tolist()
cols = cols[1:] + cols[0:1]
DF2 = DF2[cols]


# ***Chess kr-vs-kp dataset***
"""
Number of Instances: 3196
Number of Attributes: 36
Attribute values: The format for instances in this database is a sequence of 37 attribute values.
Each instance is a board-descriptions for this chess endgame.  The first
36 attributes describe the board.  The last (37th) attribute is the
classification: "win" or "nowin".  There are 0 missing values.
A typical board-description is "f,f,f,f,f,f,f,f,f,f,f,f,l,f,n,f,f,t,f,f,f,f,f,f,f,t,f,f,f,f,f,f,f,t,t,n,won"
The names of the features do not appear in the board-descriptions.
Instead, each feature correponds to a particular position in the
feature-value list.  For example, the head of this list is the value
for the feature "bkblk".  The following is the list of features, in
the order in which their values appear in the feature-value list:
[bkblk,bknwy,bkon8,bkona,bkspr,bkxbq,bkxcr,bkxwp,blxwp,bxqsq,cntxt,dsopp,dwipd,
 hdchk,katri,mulch,qxmsq,r2ar8,reskd,reskr,rimmx,rkxwp,rxmsq,simpl,skach,skewr,
 skrxp,spcop,stlmt,thrsk,wkcti,wkna8,wknck,wkovl,wkpos,wtoeg]
No missing values
"""
DF3 = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/chess/king-rook-vs-king-pawn/kr-vs-kp.data",
    header=None,
)
DF3.columns = [
    "bkblk",
    "bknwy",
    "bkon8",
    "bkona",
    "bkspr",
    "bkxbq",
    "bkxcr",
    "bkxwp",
    "blxwp",
    "bxqsq",
    "cntxt",
    "dsopp",
    "dwipd",
    "hdchk",
    "katri",
    "mulch",
    "qxmsq",
    "r2ar8",
    "reskd",
    "reskr",
    "rimmx",
    "rkxwp",
    "rxmsq",
    "simpl",
    "skach",
    "skewr",
    "skrxp",
    "spcop",
    "stlmt",
    "thrsk",
    "wkcti",
    "wkna8",
    "wknck",
    "wkovl",
    "wkpos",
    "wtoeg",
    "Class",
]


# ***Car Evaluation dataset***
"""
Number of Instances: 1728
Number of Attributes: 7
Attribute values: 
    1. buying: v-high, high, med, low
    2. maint: v-high, high, med, low
    3. doors: 2, 3, 4, 5-more
    4. persons: 2, 4, more
    5. lug_boot: small, med, big
    6. safety: low, med, high
Class Distribution: 70.023% are unacceptable, 22.222% are acceptable, 3.993% are good and 3.762% are very good
No missing values
"""
DF4 = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data",
    header=None,
)
DF4.columns = [
    "buying",
    "maint",
    "doors",
    "persons",
    "lug_boot",
    "safety",
    "Class",
]
# Makes the target classes binary - merging the acceptable, good and very good classifications
DF4["Class"] = DF4["Class"].replace(["good", "vgood"], "acc")


# ***Breast Cancer dataset***
"""
Number of Instances: 286
Number of Attributes: 10
Attribute values: 
    1. Class: no-recurrence-events, recurrence-events
    2. age: 10-19, 20-29, 30-39, 40-49, 50-59, 60-69, 70-79, 80-89, 90-99.
    3. menopause: lt40, ge40, premeno.
    4. tumor-size: 0-4, 5-9, 10-14, 15-19, 20-24, 25-29, 30-34, 35-39, 40-44,
                    45-49, 50-54, 55-59.
    5. inv-nodes: 0-2, 3-5, 6-8, 9-11, 12-14, 15-17, 18-20, 21-23, 24-26,
                    27-29, 30-32, 33-35, 36-39.
    6. node-caps: yes, no.
    7. deg-malig: 1, 2, 3.
    8. breast: left, right.
    9. breast-quad: left-up, left-low, right-up, right-low, central.
    10. irradiat:	yes, no.
Class Distribution: 70.027% are no-reccurence-events, 29.72% are recurrence-events
9 missing values in total, 8 from attribute #6 and 1 from attribute #9
"""
DF5 = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer/breast-cancer.data",
    header=None,
)
# Moves the target column from the first index to the last index
cols = DF5.columns.tolist()
cols = cols[1:] + cols[0:1]
DF5 = DF5[cols]
DF5.columns = [
    "age",
    "menopause",
    "tumor-size",
    "inv-nodes",
    "node-caps",
    "deg-malig",
    "breast",
    "breast-quad",
    "irradiat",
    "Class",
]
# Droping missing values
temp_ind1 = DF5[DF5["node-caps"] == "?"].index
temp_ind2 = DF5[DF5["breast-quad"] == "?"].index
DF5.drop(temp_ind1, inplace=True)
DF5.drop(temp_ind2, inplace=True)


# The collection of 5 datasets
DFS = {
    "Tic Tac Toe": DF1,
    "Lymphography": DF2,
    "Chess kr-vs-kp": DF3,
    "Car Evaluation": DF4,
    "Breast Cancer": DF5,
}


# Defines the evaluation metrics
METRICS = {
    "accuracy": accuracy_score,
    "precision": precision_score,
    "recall": recall_score,
    "f1_score": f1_score,
    "roc_auc_score": roc_auc_score,
}

# The methods
MODELS = ["ID3", "MyID3", "BaggingID3", "MyBaggingID3"]


# Prepares the data for modeling
def prepare_data(dataset):
    ohe = OneHotEncoder(sparse_output=False)
    lb = LabelBinarizer()
    X = dataset.iloc[:, :-1].values
    y = dataset.iloc[:, -1].values
    # One hot encoding the features
    X_bin = ohe.fit_transform(X)
    # The target feature is binarized
    y_bin = lb.fit_transform(y).ravel()

    return X_bin, y_bin


# Evaluates the models
def evaluate_models(X, y, metrics, df_name, table):
    # Initializing the figure and subplots to log
    fig, ((ax1, ax2, ax3), (ax4, ax5, ax6)) = plt.subplots(nrows=2, ncols=3, figsize=(16, 12))
    axis = [ax1, ax2, ax3, ax4, ax5, ax6]
    ax_ind = 0

    # Initializing W&B and KFold
    wandb.init(project="ID3 & BaggingID3", name=df_name, group="Report", reinit=True)
    rkf = RepeatedKFold(n_splits=5, n_repeats=2, random_state=14)

    run_times = {model_name: 0 for model_name in MODELS}
    mean_results = {model_name: [] for model_name in MODELS}
    results = {model_name: [] for model_name in MODELS}

    # For each metric evaluate all models
    for metric_name, metric_fn in metrics.items():
        for model_name in MODELS:
            start = time.time()
            # For each split
            for train_idx, test_idx in rkf.split(X):
                models = {
                    "ID3": DecisionTreeClassifier(max_depth=8, random_state=14),
                    "MyID3": MyID3(max_depth=8),
                    "BaggingID3": BaggingClassifier(
                        DecisionTreeClassifier(max_depth=8, random_state=14),
                        n_estimators=10,
                        random_state=14,
                        max_samples=0.8,
                        max_features=0.8,
                    ),
                    "MyBaggingID3": MyBaggingID3(
                        max_depth=8, n_estimators=10, max_samples=0.8, max_features=0.8
                    ),
                }
                model = models[model_name]
                # Spliting the data into train and test sets
                X_train, y_train = X[train_idx], y[train_idx]
                X_test, y_test = X[test_idx], y[test_idx]

                # Training the model on the training set
                model.fit(X_train, y_train)

                # Evaluating the model on the test set
                if metric_name == "roc_auc_score":
                    y_pred = model.predict_proba(X_test)[:, 1]
                else:
                    y_pred = model.predict(X_test)

                # Computing the evaluation metrics
                results[model_name].append(metric_fn(y_test, y_pred))
            end = time.time()

            # Computing the runtime of model on the metric in ms
            run_times[model_name] = (end - start) * 1000
            # Computing the mean of the results for each metric
            mean_results[model_name] = np.mean(results[model_name])
            # Adding the data to the report table
            table.add_data(
                df_name, model_name, metric_name, mean_results[model_name], run_times[model_name]
            )

        # Creating the plot for the metric
        colors = ["red", "orange", "blue", "cyan"]
        axis[ax_ind].bar(list(range(len(models))), list(mean_results.values()), color=colors)
        axis[ax_ind].set_ylim(
            [min(mean_results.values()) - 0.1, min(max(mean_results.values()) + 0.1, 1)]
        )
        axis[ax_ind].set_xticks(list(range(len(models))))
        axis[ax_ind].set_xticklabels(list(mean_results.keys()), rotation=30)
        axis[ax_ind].set_title(f"{metric_name} mean", fontsize=18)
        axis[ax_ind].set_ylabel("Mean Value")
        axis[ax_ind].tick_params(axis="both", which="major", labelsize=12)
        axis[ax_ind].tick_params(axis="both", which="minor", labelsize=10)
        ax_ind += 1
        # Creating the plot for the runtime
        if ax_ind == 5:
            axis[ax_ind].bar(list(range(len(models))), list(run_times.values()), color=colors)
            axis[ax_ind].set_ylim(
                [min(min(run_times.values()) - 5, 0), max(run_times.values()) + 5]
            )
            axis[ax_ind].set_xticks(list(range(len(models))))
            axis[ax_ind].set_xticklabels(list(run_times.keys()), rotation=30)
            axis[ax_ind].set_title("Run times", fontsize=18)
            axis[ax_ind].set_ylabel("ms")
            axis[ax_ind].tick_params(axis="both", which="major", labelsize=12)
            axis[ax_ind].tick_params(axis="both", which="minor", labelsize=10)
    fig.suptitle(df_name, fontsize=30)
    fig.tight_layout(pad=5.0)
    # Logging the subplots for the dataset
    wandb.log({df_name: wandb.Image(fig)})
    plt.close(fig)


# Evaluates the models per dataset
wandb.login(key="<API key here>")
report_table = wandb.Table(
    columns=["Dataset", "Method", "Evaluation metric", "Evaluation Value", "Fit Runtime (in ms)"]
)

for df_name, df in DFS.items():
    X, y = prepare_data(df)
    evaluate_models(X, y, METRICS, df_name, report_table)

# Log the report table
wandb.log({"Report Table": report_table})
wandb.finish()
